In [1]:
import json
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
import networkx as nx
import os
import time
from dataclasses import dataclass
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
from libs.lizi.my_magi import MyMagiModel
from libs.lizi.my_magi.config import MagiConfig
from libs.lizi.my_magi.utils import UnionFind
from libs.lizi.my_magi.utils import read_image_as_np_array as read_image
from numpy.typing import NDArray
from PIL import Image
from rich.pretty import pprint as pp
from torchmetrics.functional.pairwise import pairwise_cosine_similarity
from transformers.modeling_utils import load_state_dict

In [2]:
from typing import List

In [3]:
# Загрузка модели/Model initialization
state_dict = load_state_dict(str(Path("models/magi/pytorch_model.bin").resolve()))
state_dict.keys()
config: MagiConfig = MagiConfig.from_json_file(Path("libs/lizi/my_magi/config.json").resolve())  # type: ignore
model = MyMagiModel(config)
model.load_state_dict(state_dict, strict=False)
model.cuda() # type: ignore

MyMagiModel(
  (crop_embedding_model): ViTMAEModel(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTMAELayer(
          (attention): ViTMAESdpaAttention(
            (attention): ViTMAESdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_f

---

### Все dataclasses

In [4]:
# Хранение картинки страницы, как numpy массива и его названия
@dataclass
class ImageInfo:
    image: np.ndarray
    full_file_name: str

    def get_image_array(self):
        return self.image

# Для cropBox   
@dataclass
class CropBbox:
    id_crop_bbox: int
    image_bbox: np.ndarray # картиночка самого bboxa
    character_score: float # не нужен, но есть
    embeddings_for_batch: torch.Tensor # эмбеддинг для сравнения
    crop_bboxes_for: torch.Tensor # 4 координаты
    file_name: str # имя страницы исходной, иначе хрен сравним с раскраской

# Тут картинка с примером раскраски и названием всего файла с картинкой для раскраски
@dataclass
class SampleImage:
    # sample_image: int
    sample_image: np.ndarray # картиночка 
    # character_score: float # не нужен, но есть
    # embeddings_for_batch: torch.Tensor # эмбеддинг для сравнения
    # crop_bboxes_for: torch.Tensor # 4 координаты
    full_file_name: str # имя страницы исходной для подачи потом в раскраску

# Уже преобразованная картинка с эмбеддингом для сравнения
@dataclass
class AnalisysSampleImage:
    # id_sample_image: int
    sample_image: np.ndarray # картиночка 
    # character_score: float # не нужен, но есть
    embeddings_for_batch: torch.Tensor # эмбеддинг для сравнения
    # crop_bboxes_for: torch.Tensor # 4 координаты
    full_file_name: str # имя страницы исходной для подачи потом в раскраску


@dataclass
class SampleImageConnection:
    # id_sample_image: int
    crop_image_bbox: np.ndarray
    crop_bboxes_coordinates: torch.Tensor # 4 координаты
    file_page_name: str # имя страницы исходной, иначе хрен сравним с раскраской
    
    sample_image: np.ndarray # картиночка с примером раскраски
    full_sample_file_name: str # имя страницы исходной для подачи потом в раскраску

### Говнокод

In [5]:
directory_path = "data/x_manga"

In [6]:
### Uploading images
# Функция для чтания файлов из папки с целой главой
def ubpload_pages(directory_path_uploading: str) -> list[ImageInfo]:
    pages_images = []
    for filename in os.listdir(directory_path_uploading):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            full_path = os.path.join(directory_path_uploading, filename)
            try:
                img = np.asarray(Image.open(full_path).convert("RGB"))
                pages_images.append(ImageInfo(image=img, full_file_name=full_path))
            except Exception as e:
                print(f"Error when opening {full_path}: {e}")
        else:
            print(f"Incorrect file extension {directory_path_uploading}")
    return pages_images


In [7]:
# получение списка с np.ndarray, где каждый элемент - экземпляр класса ImageInfo,
# где лежит название файла страницы и сама страница в виде нампай массива
my_pages = ubpload_pages(directory_path)

In [8]:
### Get embeddings
def get_embeddings(images_pages: list[ImageInfo]) -> tuple[list[CropBbox], list[torch.Tensor]]:
    images_for_everything = []
    list_of_embedddings = []
    id_number = 0
    for batch in images_pages:
        with torch.no_grad():
            page_image = [batch.image]
            page_name = batch.full_file_name

            (
                batch_crop_bboxes,
                batch_crop_embeddings_for_batch,
                batch_image_bboxes,
                batch_character_scores,
            ) = model.get_crops_and_embeddings(page_image)

        num_rows = len(batch_crop_embeddings_for_batch[0])

        for i in range(num_rows):
            images_for_everything.append(
                CropBbox(
                    id_crop_bbox=i + id_number,
                    image_bbox=batch_image_bboxes[0][i],
                    character_score=batch_character_scores[0][i],
                    embeddings_for_batch=batch_crop_embeddings_for_batch[0][i],
                    crop_bboxes_for=batch_crop_bboxes[0][i],
                    file_name=page_name,
                )
            )
            list_of_embedddings.append(batch_crop_embeddings_for_batch[0][i])

        id_number = id_number + num_rows
    return images_for_everything, list_of_embedddings


In [9]:
list_of_bboxes, original_emb = get_embeddings(my_pages)

In [10]:
type(original_emb[0])

torch.Tensor

In [11]:
### Разделим по 100 bboxes
# Тут подсписки длиной в 100 экземпляров BBox
# sublists_for_clasterization = []
# Тут подсписки длиной в 100 тензоров каждый - тензоры для cosine_simularity
def prepreparing_embeddings(list_of_embedddings: list[torch.Tensor]) -> list[torch.Tensor]:
    sublists_for_embedddings = []
    for i in range(0, len(list_of_embedddings), 100):
        # sublist_for_bbox = images_bw_for_everything[i:i+100]
        sublist_for_embedddings = list_of_embedddings[i : i + 100]
        # sublists_for_clasterization.append(sublist_for_bbox)
        sublists_for_embedddings.append(sublist_for_embedddings)

    # Тут просто в тензоры объединим
    list_for_analysis = []
    for one_list in sublists_for_embedddings:
        crop_embedddings = None

        # Проходим по каждому элементу в подмножестве
        for i in range(len(one_list)):
            # Извлекаем эмбеддинги из текущего элемента
            current_embeddings = one_list[i].unsqueeze(dim=0)

            # Если crop_embeds еще не инициализирована, инициализируем ее текущими эмбеддингами
            if crop_embedddings is None:
                crop_embedddings = current_embeddings
            else:
                # Иначе объединяем текущие эмбеддинги с предыдущими
                crop_embedddings = torch.cat((crop_embedddings, current_embeddings), dim=0)

        list_for_analysis.append(crop_embedddings)
    return list_for_analysis

In [14]:
list_original_embeddings = prepreparing_embeddings(original_emb)

In [16]:
(list_original_embeddings[0].size())

torch.Size([86, 768])

In [17]:
### Проба с максимумом по главе
# Список для поиска совпадений с раскрашенными персами
def original_bboxes_compare(list_for_analysis: list[torch.Tensor]) -> list[torch.Tensor]:
    compare_list = []
    # Перебираем все подмножества в sublists_for_embedddings
    for one_pack_for_analysis in list_for_analysis:
        # Матрица с косинусными совпадениями все-на-все
        pcs = pairwise_cosine_similarity(one_pack_for_analysis, one_pack_for_analysis)
        # Меняем единицы в главной диагонали на нули
        pcs = pcs.fill_diagonal_(0.0)
        # Получаем индексы всех максимумов - лучшие совпадения
        new_var = torch.argmax(pcs, dim=1)
        # Объединяем индексы лучших совпадений друг с другом попарно
        char_to = torch.cat(
            (new_var.unsqueeze(1), torch.arange(len(new_var)).cuda().unsqueeze(1)), dim=1
        )
        # Делаем граф из совпадающих вершин
        graphs_chapter_one_max = nx.Graph(char_to.tolist())
        # Объединяем все совпавшие вершины друг с другом
        indixes_per_chapter = [list(c_) for c_ in nx.connected_components(graphs_chapter_one_max)]
        # Создаём список compare_list и добавляем внутри него тензоры
        for c_k in indixes_per_chapter:
            for character_index in range(len(c_k)):
                num = int(c_k[character_index])
                if character_index == 0:
                    first_compare_batch = one_pack_for_analysis[num].unsqueeze(dim=0)
                else:
                    first_compare_batch = torch.cat(
                        (first_compare_batch, one_pack_for_analysis[num].unsqueeze(dim=0)), dim=0
                    )
            compare_list.append(first_compare_batch)
    return compare_list

In [18]:
my_comp_list = original_bboxes_compare(list_original_embeddings)

In [19]:
### Поиск samples
directory_path_samples = "data/ex_samples"

In [20]:
def sample_img(
    directory_path_samples: str,
) -> tuple[torch.Tensor, list[AnalisysSampleImage]]:
    # images_samples - список, содержащий в себе примеры для окраски в виде numpy arrays
    images_samples = []
    # Заглушка для чтения файлов из папки с примерами раскраски
    for filename in os.listdir(directory_path_samples):
        full_path = os.path.join(directory_path_samples, filename)
        try:
            img = np.asarray(Image.open(full_path).convert("RGB"))
            images_samples.append(SampleImage(sample_image=img, full_file_name=full_path))
        except Exception as e:
            print(f"Ошибка при открытии {full_path}: {e}")

    images_color_for_analysis = []
    list_of_compare_embeddings = []

    # записываем все картинки для раскраски в такую же структуру, что и bboxes
    for batch in images_samples:
        with torch.no_grad():
            page_image = [batch.sample_image]
            page_name = batch.full_file_name

            (
                batch_crop_bboxes,
                batch_crop_embeddings_for_batch,
                batch_image_bboxes,
                batch_character_scores,
            ) = model.get_crops_and_embeddings(page_image)

        num_rows = len(batch_crop_embeddings_for_batch[0])

        for i in range(num_rows):
            images_color_for_analysis.append(
                AnalisysSampleImage(
                    sample_image=batch_image_bboxes[0][i],
                    embeddings_for_batch=batch_crop_embeddings_for_batch[0][i],
                    full_file_name=page_name,
                )
            )
            list_of_compare_embeddings.append(batch_crop_embeddings_for_batch[0][i])

    # Обрабатываем список с эмбеддингами - делаем из них единый тензор
    crop_embedddings_sample = None
    # Проходим по каждому элементу в подмножестве
    for i in range(len(list_of_compare_embeddings)):
        # Извлекаем эмбеддинги из текущего элемента
        current_embeddings = list_of_compare_embeddings[i].unsqueeze(dim=0)

        # Если crop_embeds_bw еще не инициализирована, инициализируем ее текущими эмбеддингами
        if crop_embedddings_sample is None:
            crop_embedddings_sample = current_embeddings
        else:
            # Иначе объединяем текущие эмбеддинги с предыдущими
            crop_embedddings_sample = torch.cat(
                (crop_embedddings_sample, current_embeddings), dim=0
            )
    return crop_embedddings_sample, images_color_for_analysis

In [21]:
my_crop_embedddings_sample, my_images_color_for_analysis = sample_img(directory_path_samples)

In [22]:
print(my_images_color_for_analysis[0].full_file_name)

data/ex_samples/0.png


In [23]:
### Сопоставляем эмбеддинги цветных примеров и эмбеддинги bboxes
def finding_samples(
    crop_embedddings_sample: torch.Tensor, compare_list: list[torch.Tensor]
) -> tuple[dict, dict]:
    # result_dict - тут будет цифра, которая привязана только к sample_dict и подходящие эмбеддинги среди нераскрашенных кропов
    result_dict = {}
    # sample_dict - тут будет цифра, которая привязана связана с result_dict и эмбеддинг раскраски
    sample_dict = {}

    for one_tensor in compare_list:
        # Составляем матрицу и сравниваем с самплами
        pcs_samples = pairwise_cosine_similarity(one_tensor, crop_embedddings_sample)
        # Ищем сумму по всем значениям
        comp = torch.sum(pcs_samples, dim=0)
        # Ищем индекс максимального совпадения
        max_coincidence = int(torch.argmax(comp))
        if result_dict.get(max_coincidence) is not None:
            inter_res = torch.cat((result_dict[max_coincidence], one_tensor), dim=0)
            result_dict[max_coincidence] = inter_res
        else:
            result_dict[max_coincidence] = one_tensor
            # Запись под аналогичным номером эмбеддинга с примером раскраски
            sample_dict[max_coincidence] = crop_embedddings_sample[max_coincidence]
    return result_dict, sample_dict

In [24]:
my_result_dict, my_sample_dict = finding_samples(my_crop_embedddings_sample, my_comp_list)

In [25]:
(my_sample_dict[0].size())

torch.Size([768])

In [26]:
(my_images_color_for_analysis[0].embeddings_for_batch.size())

torch.Size([768])

In [30]:
my_result_dict[0][0]

tensor([ 2.9781e-01,  1.9670e-02, -1.7568e-01,  2.9379e-02, -4.2561e-02,
        -9.6347e-01, -2.2351e-01, -3.4887e-01,  6.6167e-02,  2.2741e-01,
         1.0703e-01, -3.2303e-02, -2.2454e-01,  3.6913e-02,  2.0199e-01,
        -1.2524e-01,  5.2243e-02,  1.0523e-01, -1.1381e-01,  3.1429e-01,
        -1.2751e-02,  1.0706e-01,  1.0423e-02,  1.9518e-01, -7.7103e-02,
         1.4773e-01, -1.1976e-02, -1.3550e-02, -9.4123e-02, -2.7921e-02,
        -3.9593e-01, -2.0264e-02, -7.1006e-02, -3.2286e-01, -1.6692e-01,
         1.2103e-01,  5.8713e-02, -4.8541e-03, -7.1120e-02,  2.1507e-02,
        -2.8327e-03, -1.1897e-02,  1.0950e-01,  2.4325e-02, -1.0657e-01,
        -9.3130e-02,  1.3887e-01,  1.0261e-02, -1.2351e-01, -1.4700e-02,
        -5.4408e-02,  1.3477e-02,  1.9875e-01, -2.2320e-02, -3.5938e-01,
        -1.3636e-01,  7.1938e-02, -1.5723e-01, -7.5227e-01,  3.0485e-02,
         8.0218e-02, -9.4945e-02,  2.3611e-02,  2.1705e-01, -1.0553e-01,
         3.3739e-01,  5.6018e-02, -1.8904e-01,  1.1

In [41]:
def creating_pairs(
    sample_list: list[AnalisysSampleImage],
    sample_dict: dict,
    original_list: list[CropBbox],
    original_dict: dict,
) -> list[SampleImageConnection]:
    list_for_colorization = []

    def compare_tensors(tensor1, tensor2):
        return torch.allclose(tensor1, tensor2)

    for key, tensor_value in sample_dict.items():
        matching_object: AnalisysSampleImage | None = next(
            (obj for obj in sample_list if compare_tensors(obj.embeddings_for_batch, tensor_value)),
            None,
        )
        if matching_object:
            picture_sample = matching_object.sample_image
            picture_name = matching_object.full_file_name

        for j in range(original_dict[key].shape[0]):
            matching_original: CropBbox | None = next(
                (
                    obj
                    for obj in original_list
                    if compare_tensors(obj.embeddings_for_batch, original_dict[key][j])
                ),
                None,
            )
            if matching_original:
                list_for_colorization.append(
                    SampleImageConnection(
                        crop_image_bbox=matching_original.image_bbox,
                        crop_bboxes_coordinates=matching_original.crop_bboxes_for,
                        file_page_name=matching_original.file_name,
                        sample_image=picture_sample,
                        full_sample_file_name=picture_name,
                    )
                )
    return list_for_colorization


In [32]:
final_character_list = creating_pairs(
    my_images_color_for_analysis, my_sample_dict, list_of_bboxes, my_result_dict
)

---

In [ ]:
# Путь к файлу, куда вы хотите сохранить пустой JSON файл/ Надо чепм-то заменить
file_path_json = "data/json_files/Bleach/Vol74_Ch685.json"
# Путь к папке с файлами/Path to the folder with files
directory_path = "data/masi_mangas/Oshi no Ko/[Ai's fanclub] Vol. 7 Ch. 65"

manga_images = ubpload_pages(directory_path)
images_classes, list_of_all_embedddings = get_embeddings(manga_images)
prep_embeddings = prepreparing_embeddings(list_of_all_embedddings)

In [ ]:
### Uploading images
# Первая версия
# Путь к файлу, куда вы хотите сохранить пустой JSON файл/ Надо чепм-то заменить
file_path_json = "data/json_files/Bleach/Vol74_Ch685.json"
# Путь к папке с файлами/Path to the folder with files
directory_path = "data/masi_mangas/Oshi no Ko/[Ai's fanclub] Vol. 7 Ch. 65"
images_bw = []
images_color = []

# Заглушка для чтания файлов из папки с целой главой
for filename in os.listdir(directory_path):
    if filename.endswith("_bw.png"):
        full_path = os.path.join(directory_path, filename)
        try:
            img = np.asarray(Image.open(full_path).convert("RGB"))
            images_bw.append(ImageInfo(image=img, full_file_name=full_path))
        except Exception as e:
            print(f"Error when opening {full_path}: {e}")
    elif filename.endswith("_color.png"):
        full_path = os.path.join(directory_path, filename)
        try:
            img = np.asarray(Image.open(full_path).convert("RGB"))
            images_color.append(ImageInfo(image=img, full_file_name=full_path))
        except Exception as e:
            print(f"Error when opening {full_path}: {e}")

---

**image_bboxes** - list со вложенными lists, вложенный список - одна страница, где каждый элемент - np.array
**character_scores** - list с тензорами, тензор - одна страница, каждый элемент - для каждого bbox score
**crop_embeddings_for_batch** - list с тензорами, где каждый тензор - страница, а строка - для каждого bbox
**crop_bboxes** - list c тензорами, где каждый тензор - страница, а строка - для каждого bbox

In [ ]:
### Get embeddings
# Первая версия
crop_bboxes_bw = []
crop_embeddings_for_batch_bw = []
image_bboxes_bw = []
character_scores_bw = []


images_bw_for_everything_765 = []


list_of_embedddings = []

id_number = 0

for batch in images_bw:
    with torch.no_grad():
        page_image = [batch.image]
        page_name = batch.full_file_name

        (
            batch_crop_bboxes,
            batch_crop_embeddings_for_batch,
            batch_image_bboxes,
            batch_character_scores,
        ) = model.get_crops_and_embeddings(page_image)

    num_rows = len(batch_crop_embeddings_for_batch[0])

    for i in range(num_rows):
        images_bw_for_everything_765.append(
            CropBbox(
                id_crop_bbox=i+id_number,
                image_bbox=batch_image_bboxes[0][i],
                character_score=batch_character_scores[0][i],
                embeddings_for_batch=batch_crop_embeddings_for_batch[0][i],
                crop_bboxes_for=batch_crop_bboxes[0][i],
                file_name=page_name,
            )
        )
        list_of_embedddings.append(batch_crop_embeddings_for_batch[0][i])
    
    id_number = id_number + num_rows


In [ ]:
### Get embeddings


def get_embeddings(images_pages): #-> list[CropBbox], list[np.ndarray]:
    images_for_everything = []
    list_of_embedddings = []
    id_number = 0
    for batch in images_pages:
        with torch.no_grad():
            page_image = [batch.image]
            page_name = batch.full_file_name

            (
                batch_crop_bboxes,
                batch_crop_embeddings_for_batch,
                batch_image_bboxes,
                batch_character_scores,
            ) = model.get_crops_and_embeddings(page_image)

        num_rows = len(batch_crop_embeddings_for_batch[0])

        for i in range(num_rows):
            images_for_everything.append(
                CropBbox(
                    id_crop_bbox=i + id_number,
                    image_bbox=batch_image_bboxes[0][i],
                    character_score=batch_character_scores[0][i],
                    embeddings_for_batch=batch_crop_embeddings_for_batch[0][i],
                    crop_bboxes_for=batch_crop_bboxes[0][i],
                    file_name=page_name,
                )
            )
            list_of_embedddings.append(batch_crop_embeddings_for_batch[0][i])

        id_number = id_number + num_rows
    return images_for_everything, list_of_embedddings


In [ ]:
### Разделим по 100 bboxes
# Первая версия
# Тут подсписки длиной в 100 экземпляров BBox
# sublists_for_clasterization = []
# Тут подсписки длиной в 100 тензоров каждый - тензоры для cosine_simularity
sublists_for_embedddings = []
for i in range(0, len(list_of_embedddings), 100):
    #sublist_for_bbox = images_bw_for_everything[i:i+100]
    sublist_for_embedddings = list_of_embedddings[i:i+100]
    #sublists_for_clasterization.append(sublist_for_bbox)
    sublists_for_embedddings.append(sublist_for_embedddings)
# Тут просто в тензоры объединим
list_for_analysis = []
for one_list in sublists_for_embedddings:
    crop_embedddings_bw = None
    
    # Проходим по каждому элементу в подмножестве
    for i in range(len(one_list)):
        # Извлекаем эмбеддинги из текущего элемента
        current_embeddings = one_list[i].unsqueeze(dim=0)
        
        # Если crop_embeds_bw еще не инициализирована, инициализируем ее текущими эмбеддингами
        if crop_embedddings_bw is None:
            crop_embedddings_bw = current_embeddings
        else:
            # Иначе объединяем текущие эмбеддинги с предыдущими
            crop_embedddings_bw = torch.cat((crop_embedddings_bw, current_embeddings), dim=0)
    
    list_for_analysis.append(crop_embedddings_bw)

In [ ]:
### Разделим по 100 bboxes
# Тут подсписки длиной в 100 экземпляров BBox
# sublists_for_clasterization = []
# Тут подсписки длиной в 100 тензоров каждый - тензоры для cosine_simularity
def prepreparing_embeddings(list_of_embedddings) -> list[torch.Tensor]:
    sublists_for_embedddings = []
    for i in range(0, len(list_of_embedddings), 100):
        #sublist_for_bbox = images_bw_for_everything[i:i+100]
        sublist_for_embedddings = list_of_embedddings[i:i+100]
        #sublists_for_clasterization.append(sublist_for_bbox)
        sublists_for_embedddings.append(sublist_for_embedddings)

    # Тут просто в тензоры объединим
    list_for_analysis = []
    for one_list in sublists_for_embedddings:
        crop_embedddings = None
        
        # Проходим по каждому элементу в подмножестве
        for i in range(len(one_list)):
            # Извлекаем эмбеддинги из текущего элемента
            current_embeddings = one_list[i].unsqueeze(dim=0)
            
            # Если crop_embeds еще не инициализирована, инициализируем ее текущими эмбеддингами
            if crop_embedddings is None:
                crop_embedddings = current_embeddings
            else:
                # Иначе объединяем текущие эмбеддинги с предыдущими
                crop_embedddings = torch.cat((crop_embedddings, current_embeddings), dim=0)
        
        list_for_analysis.append(crop_embedddings)
    return list_for_analysis

In [ ]:
### Проба с максимумом по главе
# Список для поиска совпадений с раскрашенными персами
compare_list = []
# Перебираем все подмножества в sublists_for_embedddings
for one_pack_for_analysis in list_for_analysis:

    # Матрица с косинусными совпадениями все-на-все
    pcs = pairwise_cosine_similarity(one_pack_for_analysis, one_pack_for_analysis)
    # Меняем единицы в главной диагонали на нули
    pcs = pcs.fill_diagonal_(0.0)
    # Получаем индексы всех максимумов - лучшие совпадения
    new_var = torch.argmax(pcs, dim=1)
    # Объединяем индексы лучших совпадений друг с другом попарно
    char_to = torch.cat(
        (new_var.unsqueeze(1), torch.arange(len(new_var)).cuda().unsqueeze(1)), dim=1
    )
    # Делаем граф из совпадающих вершин
    graphs_chapter_one_max = nx.Graph(char_to.tolist())
    # Объединяем все совпавшие вершины друг с другом  
    indixes_per_chapter = [
        list(c_) for c_ in nx.connected_components(graphs_chapter_one_max)
    ]
    # Создаём список compare_list и добавляем внутри него тензоры
    for c_k in indixes_per_chapter:
        for character_index in range(len(c_k)):
            num = int(c_k[character_index])
            if character_index == 0:
                first_compare_batch = one_pack_for_analysis[num].unsqueeze(dim=0)
            else:
                first_compare_batch = torch.cat(
                    (first_compare_batch, one_pack_for_analysis[num].unsqueeze(dim=0)), dim=0
                )
        compare_list.append(first_compare_batch)

In [ ]:
### Проба с максимумом по главе
# Список для поиска совпадений с раскрашенными персами
def sample_compare(list_for_analysis):
    compare_list = []
    # Перебираем все подмножества в sublists_for_embedddings
    for one_pack_for_analysis in list_for_analysis:

        # Матрица с косинусными совпадениями все-на-все
        pcs = pairwise_cosine_similarity(one_pack_for_analysis, one_pack_for_analysis)
        # Меняем единицы в главной диагонали на нули
        pcs = pcs.fill_diagonal_(0.0)
        # Получаем индексы всех максимумов - лучшие совпадения
        new_var = torch.argmax(pcs, dim=1)
        # Объединяем индексы лучших совпадений друг с другом попарно
        char_to = torch.cat(
            (new_var.unsqueeze(1), torch.arange(len(new_var)).cuda().unsqueeze(1)), dim=1
        )
        # Делаем граф из совпадающих вершин
        graphs_chapter_one_max = nx.Graph(char_to.tolist())
        # Объединяем все совпавшие вершины друг с другом  
        indixes_per_chapter = [
            list(c_) for c_ in nx.connected_components(graphs_chapter_one_max)
        ]
        # Создаём список compare_list и добавляем внутри него тензоры
        for c_k in indixes_per_chapter:
            for character_index in range(len(c_k)):
                num = int(c_k[character_index])
                if character_index == 0:
                    first_compare_batch = one_pack_for_analysis[num].unsqueeze(dim=0)
                else:
                    first_compare_batch = torch.cat(
                        (first_compare_batch, one_pack_for_analysis[num].unsqueeze(dim=0)), dim=0
                    )
            compare_list.append(first_compare_batch)
    return compare_list

In [ ]:
### Поиск samples
directory_path_samples = "data/samples/Bleach"


# images_samples - список, содержащий в себе примеры для окраски в виде numpy arrays
images_samples = []
# Заглушка для чтения файлов из папки с примерами раскраски
for filename in os.listdir(directory_path_samples):
    full_path = os.path.join(directory_path_samples, filename)
    try:
        img = np.asarray(Image.open(full_path).convert("RGB"))
        images_samples.append(SampleImage(sample_image=img, full_file_name=full_path))
    except Exception as e:
        print(f"Ошибка при открытии {full_path}: {e}")

type(images_samples)
images_color_for_analysis = []
list_of_compare_embeddings = []

# записываем все картинки для раскраски в такую же структуру, что и bboxes
for batch in images_samples:
    with torch.no_grad():
        page_image = [batch.sample_image]
        page_name = batch.full_file_name

        (
            batch_crop_bboxes,
            batch_crop_embeddings_for_batch,
            batch_image_bboxes,
            batch_character_scores,
        ) = model.get_crops_and_embeddings(page_image)

    num_rows = len(batch_crop_embeddings_for_batch[0])

    for i in range(num_rows):
        images_color_for_analysis.append(
            AnalisysSampleImage(
                #id_sample_image=i+id_number,
                sample_image=batch_image_bboxes[0][i],
                embeddings_for_batch=batch_crop_embeddings_for_batch[0][i],
                full_file_name=page_name,
            )
        )
        list_of_compare_embeddings.append(batch_crop_embeddings_for_batch[0][i])
    
    id_number = id_number + num_rows

# Обрабатываем список с эмбеддингами - делаем из них единый тензор
crop_embedddings_sample = None
# Проходим по каждому элементу в подмножестве
for i in range(len(list_of_compare_embeddings)):
    # Извлекаем эмбеддинги из текущего элемента
    current_embeddings = list_of_compare_embeddings[i].unsqueeze(dim=0)
    
    # Если crop_embeds_bw еще не инициализирована, инициализируем ее текущими эмбеддингами
    if crop_embedddings_sample is None:
        crop_embedddings_sample = current_embeddings
    else:
        # Иначе объединяем текущие эмбеддинги с предыдущими
        crop_embedddings_sample = torch.cat((crop_embedddings_sample, current_embeddings), dim=0)


In [ ]:
### Поиск samples
directory_path_samples = "data/samples/Bleach"

def sample_img(directory_path_samples):
# images_samples - список, содержащий в себе примеры для окраски в виде numpy arrays
    images_samples = []
    # Заглушка для чтения файлов из папки с примерами раскраски
    for filename in os.listdir(directory_path_samples):
        full_path = os.path.join(directory_path_samples, filename)
        try:
            img = np.asarray(Image.open(full_path).convert("RGB"))
            images_samples.append(SampleImage(sample_image=img, full_file_name=full_path))
        except Exception as e:
            print(f"Ошибка при открытии {full_path}: {e}")

    images_color_for_analysis = []
    list_of_compare_embeddings = []

    # записываем все картинки для раскраски в такую же структуру, что и bboxes
    for batch in images_samples:
        with torch.no_grad():
            page_image = [batch.sample_image]
            page_name = batch.full_file_name

            (
                batch_crop_bboxes,
                batch_crop_embeddings_for_batch,
                batch_image_bboxes,
                batch_character_scores,
            ) = model.get_crops_and_embeddings(page_image)

        num_rows = len(batch_crop_embeddings_for_batch[0])

        for i in range(num_rows):
            images_color_for_analysis.append(
                AnalisysSampleImage(
                    sample_image=batch_image_bboxes[0][i],
                    embeddings_for_batch=batch_crop_embeddings_for_batch[0][i],
                    full_file_name=page_name,
                )
            )
            list_of_compare_embeddings.append(batch_crop_embeddings_for_batch[0][i])
        

    # Обрабатываем список с эмбеддингами - делаем из них единый тензор
    crop_embedddings_sample = None
    # Проходим по каждому элементу в подмножестве
    for i in range(len(list_of_compare_embeddings)):
        # Извлекаем эмбеддинги из текущего элемента
        current_embeddings = list_of_compare_embeddings[i].unsqueeze(dim=0)
        
        # Если crop_embeds_bw еще не инициализирована, инициализируем ее текущими эмбеддингами
        if crop_embedddings_sample is None:
            crop_embedddings_sample = current_embeddings
        else:
            # Иначе объединяем текущие эмбеддинги с предыдущими
            crop_embedddings_sample = torch.cat((crop_embedddings_sample, current_embeddings), dim=0)
    return crop_embedddings_sample, images_color_for_analysis

In [ ]:
### Сопоставляем эмбеддинги цветных примеров и эмбеддинги bboxes
# result_dict - тут будет цифра, которая привязана только к sample_dict и подходящие эмбеддинги среди нераскрашенных кропов
result_dict = {}
# sample_dict - тут будет цифра, которая привязана связана с result_dict и эмбеддинг раскраски
sample_dict = {}

for one_tensor in compare_list:
    # Составляем матрицу и сравниваем с самплами
    pcs_samples = pairwise_cosine_similarity(one_tensor, crop_embedddings_sample)
    # Ищем сумму по всем значениям
    comp = torch.sum(pcs_samples, dim=0)
    # Ищем индекс максимального совпадения
    max_coincidence = int(torch.argmax(comp))
    if result_dict.get(max_coincidence) is not None:
        inter_res = torch.cat(
                (result_dict[max_coincidence], one_tensor), dim=0)
        result_dict[max_coincidence] = inter_res
    else:
        result_dict[max_coincidence] = one_tensor
        # Запись под аналогичным номером эмбеддинга с примером раскраски
        sample_dict[max_coincidence] = crop_embedddings_sample[max_coincidence]

In [ ]:
### Сопоставляем эмбеддинги цветных примеров и эмбеддинги bboxes
def finding_samples(crop_embedddings_sample, compare_list):
    # result_dict - тут будет цифра, которая привязана только к sample_dict и подходящие эмбеддинги среди нераскрашенных кропов
    result_dict = {}
    # sample_dict - тут будет цифра, которая привязана связана с result_dict и эмбеддинг раскраски
    sample_dict = {}

    for one_tensor in compare_list:
        # Составляем матрицу и сравниваем с самплами
        pcs_samples = pairwise_cosine_similarity(one_tensor, crop_embedddings_sample)
        # Ищем сумму по всем значениям
        comp = torch.sum(pcs_samples, dim=0)
        # Ищем индекс максимального совпадения
        max_coincidence = int(torch.argmax(comp))
        if result_dict.get(max_coincidence) is not None:
            inter_res = torch.cat(
                    (result_dict[max_coincidence], one_tensor), dim=0)
            result_dict[max_coincidence] = inter_res
        else:
            result_dict[max_coincidence] = one_tensor
            # Запись под аналогичным номером эмбеддинга с примером раскраски
            sample_dict[max_coincidence] = crop_embedddings_sample[max_coincidence]
    return result_dict, sample_dict

In [ ]:
def creating_pairs(sample_list, sample_dict, original_list, original_dict):
    list_for_colorization = []
    for key, tensor_value in sample_dict.items():
        matching_object: AnalisysSampleImage | None = next(
            (obj for obj in sample_list if obj.embeddings_for_batch == tensor_value), None
        )
        if matching_object:
            picture_sample = matching_object.sample_image
            picture_name = matching_object.full_file_name

        for j in range(original_dict[key].size()):
            matching_original: CropBbox | None = next(
                (obj for obj in original_list if obj.embeddings_for_batch == original_dict[key][j]),
                None,
            )
            if matching_original:
                list_for_colorization.append(
                    SampleImageConnection(
                        crop_image_bbox=matching_original.image_bbox,
                        crop_bboxes_coordinates=matching_original.crop_bboxes_for,
                        file_page_name=matching_original.file_name,
                        sample_image=picture_sample,
                        full_sample_file_name=picture_name,
                    )
                )
